<h1 align=center><font size = 5>Segmenting and Clustering Neighborhoods in Toronto City - Part 3</font></h1>

## 1. Download required libraries

In [20]:
!conda install -c conda-forge beautifulsoup4 lxml html5lib requests geocoder geopy folium --yes

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - beautifulsoup4
    - folium
    - geocoder
    - geopy
    - html5lib
    - lxml
    - requests


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    branca-0.3.1               |             py_0          25 KB  conda-forge
    folium-0.10.0              |             py_0          59 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    altair-3.2.0               |           py36_0         770 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         882 KB

The following NEW packages will be INSTALLED:

    altair:  3.2.0-py36_0 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.10.0-py_0  conda-forge
    vincent: 0.4.4-py_1  

In [10]:
!ls

## 2. Download html page and put in a soup object

In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(website_url, 'lxml')

## 3. Scrape html page and save data into an array

In [ ]:
# Using google location
import geocoder # import geocoder

table = soup.find('table',{'class':'wikitable sortable'})
listdict = []
for row in table.find_all('tr'):
    elements = row.find_all('td')
    if not elements:
        continue
    postalCode, borough, neighborhood = [element.text.strip() for element in elements]
    if borough != 'Not assigned':
        # initialize your variable to None
        lat_lng_coords = None
        
        # loop until you get the coordinates
        while(lat_lng_coords is None):
            g = geocoder.google('{}, Toronto, Ontario'.format(postalCode))
            lat_lng_coords = g.latlng

        latitude = lat_lng_coords[0]
        longitude = lat_lng_coords[1]
        if neighborhood != 'Not assigned':
            listdict.append({'PostalCode':postalCode, 'Borough':borough, 'Neighborhood':neighborhood, 'Latitude': latitude, 'Longitude': longitude})
        else:
            listdict.append({'PostalCode':postalCode, 'Borough':borough, 'Neighborhood':borough, 'Latitude': latitude, 'Longitude': longitude})

In [11]:
# Using Geospatial_Coordinates.csv
table = soup.find('table',{'class':'wikitable sortable'})
listdict = []
coordinates = pd.read_csv('https://cocl.us/Geospatial_data', delimiter = ',')
coordinates.columns = coordinates.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
for row in table.find_all('tr'):
    elements = row.find_all('td')
    if not elements:
        continue
    postalCode, borough, neighborhood = [element.text.strip() for element in elements]
    if borough != 'Not assigned':
        latitude = coordinates[coordinates.postal_code == postalCode].latitude.iloc[0]
        longitude = coordinates[coordinates.postal_code == postalCode].longitude.iloc[0]
        if neighborhood != 'Not assigned':
            listdict.append({'PostalCode':postalCode, 'Borough':borough, 'Neighborhood':neighborhood, 'Latitude': latitude, 'Longitude': longitude})
        else:
            listdict.append({'PostalCode':postalCode, 'Borough':borough, 'Neighborhood':borough, 'Latitude': latitude, 'Longitude': longitude})

## 4. Create pandas dataframe from the array

In [12]:
# instantiate the dataframe
df = pd.DataFrame(listdict)

In [13]:
group_boroughs = df.groupby('PostalCode')['Borough'].apply(lambda x: set(x).pop())
group_neighborhoods = df.groupby('PostalCode')['Neighborhood'].apply(lambda x: "%s" % ', '.join(x))
group_df = pd.DataFrame(list(zip(group_boroughs.index, group_boroughs, group_neighborhoods)))
group_df.columns = ['PostalCode', 'Borough', 'Neighborhood']
group_df.head(10)

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [15]:
coordinates = pd.read_csv('https://cocl.us/Geospatial_data', delimiter = ',')
coordinates.columns = ['PostalCode', 'Latitude', 'Longitude']
toronto_df = pd.merge(group_df, coordinates, on='PostalCode')
toronto_df.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


In [16]:
toronto_df.shape

(103, 5)

In [17]:
toronto_df.to_csv('Toronto_Neighborhood.csv', index = False)

In [18]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(toronto_df['Borough'].unique()),
        df.shape[0]
    )
)

The dataframe has 11 boroughs and 211 neighborhoods.


#### Start Analyzing Map Data

In [21]:
# convert an address into latitude and longitude values
from geopy.geocoders import Nominatim 

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

import folium # map rendering library

In [22]:
address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto City are 43.653963, -79.387207.


In [25]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_df['Latitude'], toronto_df['Longitude'], toronto_df['Borough'], toronto_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

#### Define Foursquare Credentials and Version

In [26]:
CLIENT_ID = 'RP44N43XQD4F2UOJNABRPAICV3VW0IS1ZEJ25Y251DPJFTYI' # your Foursquare ID
CLIENT_SECRET = 'A0ETUIT10JJQYK401S1OIZBPOZZLVMCXC4S2VLV30O2JMECU' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: RP44N43XQD4F2UOJNABRPAICV3VW0IS1ZEJ25Y251DPJFTYI
CLIENT_SECRET:A0ETUIT10JJQYK401S1OIZBPOZZLVMCXC4S2VLV30O2JMECU


#### Explore the first neighborhood in the dataframe.

In [27]:
toronto_df.loc[0, 'Neighborhood']

'Rouge, Malvern'

In [28]:
neighborhood_latitude = toronto_df.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = toronto_df.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = toronto_df.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Rouge, Malvern are 43.806686299999996, -79.19435340000001.


#### Get the top 100 venues that are in Parkwoods within a radius of 500 meters.

In [29]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=RP44N43XQD4F2UOJNABRPAICV3VW0IS1ZEJ25Y251DPJFTYI&client_secret=A0ETUIT10JJQYK401S1OIZBPOZZLVMCXC4S2VLV30O2JMECU&v=20180604&ll=43.806686299999996,-79.19435340000001&radius=500&limit=100'

In [31]:
results = requests.get(url).json()

In [32]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [33]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Wendy's,Fast Food Restaurant,43.807448,-79.199056


In [34]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

1 venues were returned by Foursquare.


## 5. Explore Neighborhoods in Toronto

In [35]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [36]:
toronto_venues = getNearbyVenues(names=toronto_df['Neighborhood'],
                                 latitudes=toronto_df['Latitude'],
                                 longitudes=toronto_df['Longitude']
                                 )

Rouge, Malvern
Highland Creek, Rouge Hill, Port Union
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park, Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Clarks Corners, Sullivan, Tam O'Shanter
Agincourt North, L'Amoreaux East, Milliken, Steeles East
L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
Silver Hills, York Mills
Newtonbrook, Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park, Don Mills South
Bathurst Manor, Downsview North, Wilson Heights
Northwood Park, York University
CFB Toronto, Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens, Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West, 

In [37]:
print(toronto_venues.shape)
toronto_venues.head()

(2268, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge, Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Chris Effects Painting,43.784343,-79.163742,Construction & Landscaping
2,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
3,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store


In [38]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Agincourt,4,4,4,4,4,4
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",2,2,2,2,2,2
"Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown",10,10,10,10,10,10
"Alderwood, Long Branch",9,9,9,9,9,9
"Bathurst Manor, Downsview North, Wilson Heights",19,19,19,19,19,19
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",23,23,23,23,23,23
Berczy Park,57,57,57,57,57,57


In [39]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 275 uniques categories.


## 6. Analyze Each Neighborhood

In [40]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [41]:
toronto_onehot.shape

(2268, 275)

In [42]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,"Adelaide, King, Richmond",0.000000,0.0,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,...,0.000000,0.0,0.020000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.0
1,Agincourt,0.000000,0.0,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
2,"Agincourt North, L'Amoreaux East, Milliken, St...",0.000000,0.0,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",0.000000,0.0,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
4,"Alderwood, Long Branch",0.000000,0.0,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
5,"Bathurst Manor, Downsview North, Wilson Heights",0.000000,0.0,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,...,0.000000,0.0,0.000000,0.000000,0.052632,0.000000,0.000000,0.000000,0.000000,0.0
6,Bayview Village,0.000000,0.0,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
7,"Bedford Park, Lawrence Manor East",0.000000,0.0,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
8,Berczy Park,0.000000,0.0,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,...,0.000000,0.0,0.017544,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
9,"Birch Cliff, Cliffside West",0.000000,0.0,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0


In [43]:
toronto_grouped.shape

(100, 275)

In [44]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
            venue  freq
0     Coffee Shop  0.08
1            Café  0.05
2      Steakhouse  0.04
3             Bar  0.04
4  Cosmetics Shop  0.03


----Agincourt----
            venue  freq
0          Lounge  0.25
1  Breakfast Spot  0.25
2    Skating Rink  0.25
3  Clothing Store  0.25
4     Yoga Studio  0.00


----Agincourt North, L'Amoreaux East, Milliken, Steeles East----
                 venue  freq
0           Playground   0.5
1                 Park   0.5
2          Yoga Studio   0.0
3   Mexican Restaurant   0.0
4  Monument / Landmark   0.0


----Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown----
                  venue  freq
0           Pizza Place   0.2
1         Grocery Store   0.2
2              Pharmacy   0.1
3  Fast Food Restaurant   0.1
4        Sandwich Place   0.1


----Alderwood, Long Branch----
          venue  freq
0   Pizza Place  0.22
1  Skating Rink  0.11
2   Coffee Shop  0.

                        venue  freq
0                      Bakery  0.25
1  Construction & Landscaping  0.25
2                        Park  0.25
3               Deli / Bodega  0.25
4                 Yoga Studio  0.00


----East Birchmount Park, Ionview, Kennedy Park----
               venue  freq
0     Discount Store   0.4
1        Bus Station   0.2
2   Department Store   0.2
3        Coffee Shop   0.2
4  Mobile Phone Shop   0.0


----East Toronto----
                       venue  freq
0                Coffee Shop  0.33
1          Convenience Store  0.33
2                       Park  0.33
3                Yoga Studio  0.00
4  Middle Eastern Restaurant  0.00


----Emery, Humberlea----
                             venue  freq
0           Furniture / Home Store   0.5
1                   Baseball Field   0.5
2                      Yoga Studio   0.0
3        Middle Eastern Restaurant   0.0
4  Molecular Gastronomy Restaurant   0.0


----Fairview, Henry Farm, Oriole----
                  venue

                             venue  freq
0                       Playground   1.0
1                      Yoga Studio   0.0
2               Mexican Restaurant   0.0
3              Monument / Landmark   0.0
4  Molecular Gastronomy Restaurant   0.0


----St. James Town----
                venue  freq
0         Coffee Shop  0.07
1                Café  0.06
2          Restaurant  0.05
3               Hotel  0.05
4  Italian Restaurant  0.04


----Stn A PO Boxes 25 The Esplanade----
                venue  freq
0         Coffee Shop  0.11
1  Italian Restaurant  0.04
2          Restaurant  0.04
3                Café  0.04
4  Seafood Restaurant  0.03


----Studio District----
                 venue  freq
0                 Café  0.11
1          Coffee Shop  0.08
2  American Restaurant  0.05
3               Bakery  0.05
4   Italian Restaurant  0.05


----The Annex, North Midtown, Yorkville----
            venue  freq
0     Coffee Shop  0.12
1            Café  0.12
2  Sandwich Place  0.12
3     Piz

#### Put that into a *pandas* dataframe

In [45]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [46]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Bar,Steakhouse,Restaurant,Hotel,Cosmetics Shop,Burger Joint,Thai Restaurant,American Restaurant
1,Agincourt,Lounge,Skating Rink,Clothing Store,Breakfast Spot,Event Space,Falafel Restaurant,Farmers Market,Ethiopian Restaurant,Electronics Store,Dim Sum Restaurant
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Park,Playground,Women's Store,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Pizza Place,Grocery Store,Pharmacy,Fried Chicken Joint,Coffee Shop,Sandwich Place,Beer Store,Fast Food Restaurant,Gift Shop,Curling Ice
4,"Alderwood, Long Branch",Pizza Place,Gym,Pool,Coffee Shop,Pharmacy,Skating Rink,Pub,Sandwich Place,Dessert Shop,Dim Sum Restaurant


## 7. Cluster Neighborhoods Using K Mean

In [47]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [48]:
# neighborhoods_venues_sorted.head(10)

neighborhoods_venues_sorted[ neighborhoods_venues_sorted['Neighborhood'] == 'Islington']

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


In [49]:
toronto_merged = toronto_df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood').astype(object)

toronto_merged.head() # check the last columns!

toronto_merged.fillna(value={'Cluster Labels':0}, inplace=True)

In [50]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 8. Examine Clusters

#### Cluster 1

In [51]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Scarborough,0.0,Construction & Landscaping,Bar,Women's Store,Dumpling Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant
2,Scarborough,0.0,Mexican Restaurant,Rental Car Location,Electronics Store,Pizza Place,Intersection,Breakfast Spot,Medical Center,Women's Store,Diner,Discount Store
3,Scarborough,0.0,Coffee Shop,Korean Restaurant,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Women's Store
4,Scarborough,0.0,Hakka Restaurant,Thai Restaurant,Athletics & Sports,Bakery,Bank,Caribbean Restaurant,Fried Chicken Joint,Eastern European Restaurant,Dumpling Restaurant,Drugstore
6,Scarborough,0.0,Discount Store,Bus Station,Department Store,Coffee Shop,Drugstore,Diner,Dog Run,Doner Restaurant,Donut Shop,Women's Store
7,Scarborough,0.0,Bus Line,Bakery,Park,Fast Food Restaurant,Metro Station,Intersection,Bus Station,Soccer Field,Coworking Space,Comic Shop
8,Scarborough,0.0,Motel,Movie Theater,American Restaurant,Dessert Shop,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
9,Scarborough,0.0,College Stadium,Café,Skating Rink,General Entertainment,Donut Shop,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore
10,Scarborough,0.0,Indian Restaurant,Vietnamese Restaurant,Latin American Restaurant,Chinese Restaurant,Pet Store,Comfort Food Restaurant,Comic Shop,Falafel Restaurant,Event Space,Ethiopian Restaurant
11,Scarborough,0.0,Smoke Shop,Middle Eastern Restaurant,Bakery,Breakfast Spot,Women's Store,Drugstore,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant


#### Cluster 2

In [52]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
98,York,1.0,Park,Women's Store,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant


#### Cluster 3

In [53]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Scarborough,2.0,Playground,Women's Store,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
14,Scarborough,2.0,Park,Playground,Women's Store,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop


#### Cluster 4

In [54]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,3.0,Fast Food Restaurant,Women's Store,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant


#### Cluster 5

In [55]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,North York,4.0,Park,Bank,Convenience Store,Women's Store,Dumpling Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
25,North York,4.0,Food & Drink Shop,Park,Women's Store,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
30,North York,4.0,Airport,Park,Women's Store,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
40,East York,4.0,Convenience Store,Coffee Shop,Park,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
44,Central Toronto,4.0,Bus Line,Park,Swim School,Women's Store,Donut Shop,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore
50,Downtown Toronto,4.0,Park,Trail,Playground,Building,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant
64,Central Toronto,4.0,Trail,Jewelry Store,Park,Sushi Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
72,North York,4.0,Park,Pizza Place,Japanese Restaurant,Pub,Women's Store,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Dog Run
74,York,4.0,Park,Women's Store,Market,Fast Food Restaurant,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dessert Shop
79,North York,4.0,Deli / Bodega,Park,Bakery,Construction & Landscaping,Women's Store,Drugstore,Discount Store,Dog Run,Doner Restaurant,Donut Shop
